# Lógica

Este cuaderno de Jupyter actúa como material de apoyo para los temas tratados en el __Capítulo 6 Agentes lógicos__, el __Capítulo 7 Lógica de primer orden__ y el __Capítulo 8 Inferencia en lógica de primer orden__ del libro *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. Hacemos uso de las implementaciones en el módulo [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu). Consulte [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu) para obtener instrucciones.

Primero importemos todo desde el módulo "lógica".

In [1]:
from utils import *
from logic import *
from notebook import psource

## CONTENIDO
- Oraciones lógicas
- Expr.
- PropKB
- Agentes basados ​​en el conocimiento
- Inferencia en base de conocimiento proposicional
- Enumeración de la tabla de verdad.
- Prueba por resolución
- Encadenamiento hacia adelante y hacia atrás
-DPLL
- CaminarSAT
- Plan SAT
-FolKB
- Inferencia en base de conocimientos de primer orden.
- Unificación
- Algoritmo de encadenamiento directo
- Algoritmo de encadenamiento hacia atrás

## Oraciones lógicas

La clase `Expr` está diseñada para representar cualquier tipo de expresión matemática. El tipo más simple de `Expr` es un símbolo, que se puede definir con la función `Symbol`:

In [2]:
Symbol('x')

x

O podemos definir múltiples símbolos al mismo tiempo con la función `symbols`:

In [3]:
(x, y, P, Q, f) = symbols('x, y, P, Q, f')

Podemos combinar `Expr` con los operadores de prefijo y infijo habituales de Python. Así es como formaríamos la oración lógica "P y no Q":

In [4]:
P & ~Q

(P & ~Q)

Esto funciona porque la clase `Expr` sobrecarga el operador `&` con esta definición:

```python
def __and__(self, other): return Expr('&',  self, other)
```

y realiza sobrecargas similares para los otros operadores. Un `Expr` tiene dos campos: `op` para el operador, que siempre es una cadena, y `args` para los argumentos, que es una tupla de 0 o más expresiones. Por "expresión" me refiero a una instancia de "Expr" o a un número. Echemos un vistazo a los campos para ver algunos ejemplos de `Expr`:

In [5]:
sentence = P & ~Q

sentence.op
sentence.args
P.op
P.args
Pxy = P(x, y)

Pxy.op
Pxy.args

'&'

Es importante señalar que la clase `Expr` no define la *lógica* de las oraciones de Lógica Proposicional; simplemente te da una manera de *representar* expresiones. Piense en un `Expr` como un [abstract syntax tree](https://en.wikipedia.org/wiki/Abstract_syntax_tree). Cada uno de los "argumentos" en un "Expr" puede ser un símbolo, un número o un "Expr" anidado. Podemos anidar estos árboles a cualquier profundidad. Aquí hay un `Expr` profundamente anidado:

In [11]:
3 * f(x, y) + P(y) / 2 + 1

(((3 * f(x, y)) + (P(y) / 2)) + 1)

## Operadores para construir oraciones lógicas

A continuación se muestra una tabla de los operadores que se pueden utilizar para formar oraciones. Tenga en cuenta que tenemos un problema: queremos usar operadores de Python para crear oraciones, de modo que nuestros programas (y nuestras sesiones interactivas como la que aparece aquí) muestren código simple. Pero Python no permite flechas de implicación como operadores, por lo que por ahora tenemos que usar una notación más detallada que Python sí permite: `|'==>'|` en lugar de simplemente `==>`. Alternativamente, siempre puedes usar las formas de constructor `Expr` más detalladas:

| Operación | Libro | Entrada infija de Python | Salida de Python | Entrada `Expr` de Python
|--------------------------|----------------------|-------------------------|---|---|
| Negación | ¬ P | `~P` | `~P` | `Expr('~', P)`
| Y | PAG ∧ Q | `P y Q` | `P y Q` | `Expr('&', P, Q)`
| O | PAG ∨ Q | `P`<tt> | </tt>`Q`| `P`<tt> | </tt>`Q` | `Expr('`|`', P, Q)`
| Desigualdad (Xor) | PAG ≠ Q | `P^Q` | `P^Q` | `Expr('^', P, Q)`
| Implicación | P → Q | `P` <tt>|</tt>`'==>'`<tt>|</tt> `Q` | `P ==> Q` | `Expr('==>', P, Q)`
| Implicación inversa | Q ← PAG | `Q` <tt>|</tt>`'<=='`<tt>|</tt> `P` |`Q <== P` | `Expr('<==', Q, P)`
| Equivalencia | P ↔ Q | `P` <tt>|</tt>`'<=>'`<tt>|</tt> `Q` |`P <=> Q` | `Expr('<=>', P, Q)`

A continuación se muestra un ejemplo de cómo definir una oración con una flecha de implicación:

In [12]:
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

## `expr`: un atajo para construir oraciones

Si la notación `|'==>'|` te parece fea, puedes usar la función `expr` en su lugar:

In [13]:
expr('~(P & Q)  ==>  (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

`expr` toma una cadena como entrada y la analiza en un `Expr`. La cadena puede contener operadores de flecha: `==>`, `<==` o `<=>`, que se manejan como si fueran operadores infijos normales de Python. Y `expr` define automáticamente cualquier símbolo, por lo que no es necesario predefinirlo:

In [14]:
expr('sqrt(b ** 2 - 4 * a * c)')

sqrt(((b ** 2) - ((4 * a) * c)))

Por ahora eso es todo lo que necesitas saber sobre `expr`. Si está interesado, explicamos los complicados detalles de cómo se implementa `expr` y cómo se maneja `|'==>'|` en el apéndice.

## Bases de conocimiento proposicionales: `PropKB`

La clase `PropKB` se puede utilizar para representar una base de conocimiento de oraciones lógicas proposicionales.

Vemos que la clase `KB` tiene cuatro métodos, además de `__init__`. Un punto a tener en cuenta aquí: el método `ask` simplemente llama al método `ask_generator`. Por lo tanto, ésta ya ha sido implementada, y lo que realmente tendrás que implementar cuando crees tu propia clase de base de conocimientos (aunque probablemente nunca necesitarás hacerlo, considerando las que hemos creado para ti) será la clase ` función Ask_generator` y no la función `ask` en sí.

La clase `PropKB` ahora.
* `__init__(self, frase=None)`: El constructor `__init__` crea un único campo `cláusulas` que será una lista de todas las oraciones de la base de conocimiento. Tenga en cuenta que cada una de estas oraciones será una "cláusula", es decir, una oración que se compone únicamente de literales y "o".
* `tell(self, frase)`: cuando desea agregar una oración a la base de conocimiento, utiliza el método `tell`. Este método toma una oración, la convierte a su CNF, extrae todas las cláusulas y las agrega al campo "cláusulas". Por lo tanto, no necesita preocuparse por "decir" sólo cláusulas a la base de conocimientos. Puede "decirle" a la base de conocimientos una oración en cualquier forma que desee; convertirlo a CNF y agregar las cláusulas resultantes será manejado por el método `tell`.
* `ask_generator(self, query)`: la función `ask_generator` es utilizada por la función `ask`. Llama a la función `tt_entails`, que a su vez devuelve `True` si la base de conocimientos implica una consulta y `False` en caso contrario. El propio `ask_generator` devuelve un dictado vacío `{}` si la base de conocimientos implica una consulta y `Ninguno` en caso contrario. Esto puede parecerte un poco extraño. Después de todo, tiene más sentido simplemente devolver un "Verdadero" o un "Falso" en lugar de "{}" o "Ninguno". Pero esto se hace para mantener la coherencia con la forma en que son las cosas en la lógica de primer orden, donde un Se supone que la función `ask_generator` devuelve todas las sustituciones que hacen que la consulta sea verdadera. De ahí la sentencia, devolver todas estas sustituciones. Principalmente usaré la función `ask` que devuelve un `{}` o un `False`, pero si no te gusta esto, siempre puedes usar la función `ask_if_true` que devuelve un `True` o un "Falso".
* `retract(self, frase)`: Esta función elimina todas las cláusulas de la frase dada, de la base de conocimientos. Al igual que la función `tell`, no es necesario pasar cláusulas para eliminarlas de la base de conocimientos; cualquier frase servirá. La función se encargará de convertir esa oración en cláusulas y luego eliminarlas.

## Mundo Wumpus KB
Creemos un `PropKB` para el mundo wumpus con las oraciones mencionadas en la `sección 7.4.3`.

In [15]:
wumpus_kb = PropKB()

Definimos los símbolos que utilizamos en nuestras cláusulas.<br/>
$P_{x, y}$ es verdadero si hay un hoyo en `[x, y]`.<br/>
$B_{x, y}$ es verdadero si el agente siente brisa en `[x, y]`.<br/>

In [16]:
P11, P12, P21, P22, P31, B11, B21 = expr('P11, P12, P21, P22, P31, B11, B21')

Ahora contamos oraciones basadas en la `sección 7.4.3`.<br/>
No hay ningún hoyo en "[1,1]".

In [17]:
wumpus_kb.tell(~P11)

Una plaza tiene brisa si y sólo si hay un hoyo en una plaza vecina. Esto debe indicarse para cada cuadrado, pero por ahora incluimos sólo los cuadrados relevantes.

In [18]:
wumpus_kb.tell(B11 | '<=>' | ((P12 | P21)))
wumpus_kb.tell(B21 | '<=>' | ((P11 | P22 | P31)))

Ahora incluimos las percepciones de brisa para los dos primeros cuadrados que conducen a la situación de la "Figura 7.3(b)".

In [19]:
wumpus_kb.tell(~B11)
wumpus_kb.tell(B21)

Podemos comprobar las cláusulas almacenadas en una `KB` accediendo a su variable `cláusulas`

In [20]:
wumpus_kb.clauses

[~P11,
 (~P12 | B11),
 (~P21 | B11),
 (P12 | P21 | ~B11),
 (~P11 | B21),
 (~P22 | B21),
 (~P31 | B21),
 (P11 | P22 | P31 | ~B21),
 ~B11,
 B21]

Vemos que la equivalencia $B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ se convirtió automáticamente en dos implicaciones que a su vez se convirtieron a CNF que se almacena en el `KB `.<br/>
$B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ se dividió en $B_{1, 1} \implica (P_{1, 2} \lor P_{2) , 1})$ y $B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$.<br/>
$B_{1, 1} \implica (P_{1, 2} \lor P_{2, 1})$ se convirtió en $P_{1, 2} \lor P_{2, 1} \lor \neg B_{ 1, 1}$.<br/>
$B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$ se convirtió en $\neg (P_{1, 2} \lor P_{2, 1}) \lor B_{1, 1}$ que se convierte en $(\neg P_{1, 2} \lor B_{1, 1}) \land (\neg P_{2, 1} \lor B_{1, 1})$ después aplicando las leyes de De Morgan y distribuyendo la disyunción.<br/>
$B_{2, 1} \iff (P_{1, 1} \lor P_{2, 2} \lor P_{3, 2})$ se convierte de manera similar.

## Agentes basados ​​en el conocimiento

Un agente basado en conocimiento es un agente genérico simple que mantiene y maneja una base de conocimiento.
La base de conocimientos puede contener inicialmente algunos conocimientos previos.
<br>
El propósito de un agente de KB es proporcionar un nivel de abstracción sobre la manipulación de la base de conocimientos y debe usarse como clase base para agentes que trabajan en una base de conocimientos.
<br>
Dado un percepto, el agente KB agrega el percepto a su base de conocimientos, pregunta a la base de conocimientos cuál es la mejor acción y le dice a la base de conocimientos que, de hecho, ha realizado esa acción.
<br>
Nuestra implementación de `KB-Agent` está encapsulada en una clase `KB_AgentProgram` que hereda de la clase `KB`.
<br>
Echemos un vistazo.

In [21]:
psource(KB_AgentProgram)

Las funciones auxiliares `make_percept_sentence`, `make_action_query` y `make_action_sentence` tienen todos los nombres adecuados y son como se esperaba.
`make_percept_sentence` crea oraciones lógicas de primer orden sobre percepciones que queremos que reciba nuestro agente,
`make_action_query` pregunta al `KB` subyacente sobre la acción que se debe tomar y
`make_action_sentence` le dice al `KB` subyacente sobre la acción que acaba de realizar.

## Inferencia en la base de conocimientos proposicional
En esta sección veremos dos algoritmos para comprobar si una oración está implicada por "KB". Nuestro objetivo es decidir si $\text{KB} \vDash \alpha$ para alguna oración $\alpha$.
### Enumeración de la tabla de verdad
Es un enfoque de verificación de modelos que, como sugiere el nombre, enumera todos los modelos posibles en los que "KB" es verdadero y verifica si $\alpha$ también es verdadero en estos modelos. Enumeramos los símbolos $n$ en `KB` y enumeramos los modelos $2^{n}$ primero en profundidad y verificamos la veracidad de `KB` y $\alpha$.

In [22]:
psource(tt_check_all)

Básicamente, el algoritmo calcula cada línea de la tabla de verdad $KB\implica \alpha$ y comprueba si es cierta en todas partes.
<br>
Si se definen símbolos, la rutina construye recursivamente cada combinación de valores de verdad para los símbolos y luego,
comprueba si el "modelo" es coherente con "kb".
Los modelos dados corresponden a las líneas de la tabla de verdad,
que tienen un "verdadero" en la columna KB,
y para estas líneas verifica si la consulta se evalúa como verdadera
<br>
`resultado = pl_true(alfa, modelo)`.
<br>
<br>
En resumen, `tt_check_all` evalúa esta expresión lógica para cada `modelo`
<br>
`pl_true(kb, modelo) => pl_true(alfa, modelo)`
<br>
que es lógicamente equivalente a
<br>
`pl_true(kb, modelo) & ~pl_true(alfa, modelo)`
<br>
es decir, la base de conocimientos y la negación de la consulta son lógicamente inconsistentes.
<br>
<br>
`tt_entails()` simplemente extrae los símbolos de la consulta y llama a `tt_check_all()` con los parámetros adecuados.


In [23]:
psource(tt_entails)

Tenga en cuenta que para dos símbolos P y Q, P => Q es falso solo cuando P es "Verdadero" y Q es "Falso".
Ejemplo de uso de `tt_entails()`:

In [24]:
tt_entails(P & Q, Q)

True

P y Q son verdaderos sólo cuando P y Q son verdaderos. Por lo tanto, (P & Q) => Q es Verdadero

In [25]:
tt_entails(P | Q, Q)

False

In [26]:
tt_entails(P | Q, P)

False

Si sabemos que P | Q es verdadera, no podemos inferir los valores de verdad de P y Q.
Por lo tanto (P | Q) => Q es falso y también lo es (P | Q) => P.

In [27]:
(A, B, C, D, E, F, G) = symbols('A, B, C, D, E, F, G')
tt_entails(A & (B | C) & D & E & ~(F | G), A & D & E & ~F & ~G)

True

Podemos ver que para que la KB sea verdadera, A, D, E tienen que ser Verdaderas y F y G tienen que ser Falsas.
No se puede decir nada sobre B o C.

Volviendo a nuestro problema, tenga en cuenta que `tt_entails()` toma un `Expr` que es una conjunción de cláusulas como entrada en lugar del `KB` mismo.
Puede utilizar el método `ask_if_true()` de `PropKB` que realiza todas las conversiones necesarias.
Veamos qué nos dice `wumpus_kb` sobre $P_{1, 1}$.

In [28]:
wumpus_kb.ask_if_true(~P11), wumpus_kb.ask_if_true(P11)

(True, False)

Si observamos la Figura 7.9, vemos que en todos los modelos en los que la base de conocimiento es "Verdadera", $P_{1, 1}$ es "Falso". Tiene sentido que `ask_if_true()` devuelva `True` para $\alpha = \neg P_{1, 1}$ y `False` para $\alpha = P_{1, 1}$. Esto plantea la pregunta: ¿qué pasa si $\alpha$ es "Verdadero" sólo en una parte de todos los modelos? ¿Devolvemos "Verdadero" o "Falso"? Esto no descarta la posibilidad de que $\alpha$ sea "Verdadero", pero no está incluido en la "KB", por lo que devolvemos "Falso" en tales casos. Podemos ver que este es el caso de $P_{2, 2}$ y $P_{3, 1}$.

In [29]:
wumpus_kb.ask_if_true(~P22), wumpus_kb.ask_if_true(P22)

(False, False)

### Prueba por Resolución
Recuerde que nuestro objetivo es comprobar si $\text{KB} \vDash \alpha$ es decir, es $\text{KB} \implies \alpha$ verdadero en cada modelo. Supongamos que queremos comprobar si $P \implica Q$ es válido. Comprobamos la satisfacibilidad de $\neg (P \implica Q)$, que puede reescribirse como $P \land \neg Q$. Si $P \land \neg Q$ es insatisfactorio, entonces $P \implica Q$ debe ser verdadero en todos los modelos. Esto nos da el resultado "$\text{KB} \vDash \alpha$ <em>si y sólo si</em> $\text{KB} \land \neg \alpha$ es insatisfactorio".<br/>
Esta técnica corresponde a la <em>prueba por <strong>contradicción</strong></em>, una técnica de prueba matemática estándar. Asumimos que $\alpha$ es falso y demostramos que esto conduce a una contradicción con axiomas conocidos en $\text{KB}$. Obtenemos una contradicción haciendo inferencias válidas usando reglas de inferencia. En esta prueba usamos una única regla de inferencia, <strong>resolución</strong> que establece $(l_1 \lor \dots \lor l_k) \land (m_1 \lor \dots \lor m_n) \land (l_i \iff \ neg m_j) \implica l_1 \lor \dots \lor l_{i - 1} \lor l_{i + 1} \lor \dots \lor l_k \lor m_1 \lor \dots \lor m_{j - 1} \lor m_{j + 1} \lor \dots \lor m_n$. La aplicación de la resolución nos produce una cláusula que agregamos a la KB. Seguimos haciendo esto hasta:

* No hay cláusulas nuevas que se puedan agregar, en cuyo caso $\text{KB} \nvDash \alpha$.
* Dos cláusulas se resuelven para producir la <em>cláusula vacía</em>, en cuyo caso $\text{KB} \vDash \alpha$.

La <em>cláusula vacía</em> equivale a <em>Falso</em> porque surge únicamente de resolver dos complementarias
cláusulas unitarias como $P$ y $\neg P$, lo cual es una contradicción ya que tanto $P$ como $\neg P$ no pueden ser <em>Verdadero</em> al mismo tiempo.

Sin embargo, hay un problema: el algoritmo que implementa la prueba por resolución no puede manejar oraciones complejas.
Las implicaciones y bi-implicaciones deben simplificarse en cláusulas más simples.
Ya sabemos que *cada oración de una lógica proposicional es lógicamente equivalente a una conjunción de cláusulas*.
Usaremos este hecho a nuestro favor y simplificaremos la oración de entrada a la **forma normal conjuntiva** (CNF), que es una conjunción de disyunciones de literales.
Para mí:
<br>
$$(A\lor B)\land (\neg B\lor C\lor\neg D)\land (D\lor\neg E)$$
Esto equivale a la forma POS (Producto de sumas) en electrónica digital.
<br>
A continuación se muestra un resumen de cómo se realiza la conversión:
1. Convertir bi-implicaciones en implicaciones
<br>
$\alpha\iff\beta$ se puede escribir como $(\alpha\implies\beta)\land(\beta\implies\alpha)$
<br>
Esto también se aplica a oraciones compuestas.
<br>
$\alpha\iff(\beta\lor\gamma)$ se puede escribir como $(\alpha\implies(\beta\lor\gamma))\land((\beta\lor\gamma)\implies\alpha)$
<br>
2. Convertir implicaciones a sus equivalentes lógicos.
<br>
$\alpha\implica\beta$ se puede escribir como $\neg\alpha\lor\beta$
<br>
3. Mueva la negación hacia adentro
<br>
CNF requiere literales atómicos. Por tanto, la negación no puede aparecer en un enunciado compuesto.
Las leyes de De Morgan serán útiles aquí.
<br>
$\neg(\alpha\and\beta)\equiv(\neg\alpha\lore\neg\beta)$
<br>
$\neg(\alpha\o\beta)\equiv(\neg\alphaland\neg\beta)$
<br>
4. Distribuir disyunción sobre conjunción.
<br>
La disyunción y la conjunción son distributivas entre sí.
Ahora que solo tenemos conjunciones, disyunciones y negaciones en nuestra expresión,
Distribuiremos disyunciones sobre conjunciones siempre que sea posible, ya que esto nos dará una oración que es una conjunción de cláusulas más simples.
que es lo que queríamos en primer lugar.
<br>
Necesitamos un término de la forma
<br>
$(\alpha_ {1}\lor\alpha_ {2}\lor\alpha_ {3}...)\land(\beta_ {1}\lor\beta_ {2}\lor\beta_ {3}... )\land(\gamma_{1}\lor\gamma_{2}\lor\gamma_{3}...)\land...$
<br>
<br>
La función `to_cnf` ejecuta esta conversión utilizando subrutinas auxiliares.

In [30]:
psource(to_cnf)

`to_cnf` llama a tres subrutinas.
<br>
`eliminate_implications` convierte bi-implicaciones e implicaciones en sus equivalentes lógicos.
<br>
`move_not_inwards` elimina las negaciones de declaraciones compuestas y las mueve hacia adentro usando las leyes de De Morgan.
<br>
`distribute_and_over_or` distribuye disyunciones sobre conjunciones.
<br>
Ejecute la siguiente celda para obtener detalles de implementación.

In [31]:
psource(eliminate_implications)
psource(move_not_inwards)
psource(distribute_and_over_or)

Convirtamos algunas oraciones para ver cómo funciona.


In [32]:
A, B, C, D = expr('A, B, C, D')
to_cnf(A |'<=>'| B)

((A | ~B) & (B | ~A))

In [33]:
to_cnf(A |'<=>'| (B & C))

((A | ~B | ~C) & (B | ~A) & (C | ~A))

In [34]:
to_cnf(A & (B | (C & D)))

(A & (C | B) & (D | B))

In [35]:
to_cnf((A |'<=>'| ~B) |'==>'| (C | ~D))

((B | ~A | C | ~D) & (A | ~A | C | ~D) & (B | ~B | C | ~D) & (A | ~B | C | ~D))

Volviendo a nuestro problema de resolución, podemos ver cómo se utiliza aquí la función `to_cnf`

In [36]:
psource(pl_resolution)

In [37]:
pl_resolution(wumpus_kb, ~P11), pl_resolution(wumpus_kb, P11)

(True, False)

In [38]:
pl_resolution(wumpus_kb, ~P22), pl_resolution(wumpus_kb, P22)

(False, False)

### Encadenamiento hacia adelante y hacia atrás
Anteriormente, dijimos que veremos dos algoritmos para verificar si una oración está implicada por "KB". Aquí hay un tercero.
La diferencia aquí es que nuestro objetivo ahora es determinar si una base de conocimiento de cláusulas definidas implica un único símbolo de proposición *q*: la consulta.
Sin embargo, hay un problema: la base de conocimientos solo puede contener **cláusulas Horn**.
<br>
#### Cláusulas de cuerno
Las cláusulas de cuerno se pueden definir como una *disyunción* de *literales* con **como máximo** un literal positivo.
<br>
Una cláusula de Horn con exactamente un literal positivo se llama *cláusula definida*.
<br>
Una cláusula de Horn podría parecerse
<br>
$\neg a\lor\neg b\lor\neg c\lor\neg d... \lor z$
<br>
Esta, casualmente, también es una cláusula definida.
<br>
Utilizando las leyes de De Morgan, el ejemplo anterior se puede simplificar a
<br>
$a\land b\land c\land d ... \implica z$
<br>
Esto parece una representación lógica de cómo los humanos procesan datos y hechos conocidos.
Suponiendo que las percepciones `a`, `b`, `c`, `d`... sean verdaderas simultáneamente, podemos inferir que `z` también es verdadera en ese momento.
Hay algunos aspectos interesantes de las cláusulas de Horn que facilitan la inferencia o *resolución* algorítmica.
- Las cláusulas definidas se pueden escribir como implicaciones:
<br>
La simplificación más importante que proporciona una cláusula definida es que puede escribirse como una implicación.
La premisa (o el conocimiento que conduce a la implicación) es una conjunción de literales positivos.
La conclusión (la declaración implícita) también es un literal positivo.
La frase se vuelve así más fácil de entender.
La premisa y la conclusión se denominan convencionalmente *cuerpo* y *cabeza* respectivamente.
Un único literal positivo se llama *hecho*.
- El encadenamiento hacia adelante y el encadenamiento hacia atrás se pueden utilizar para realizar inferencias a partir de cláusulas de Horn:
<br>
El encadenamiento directo es semánticamente idéntico a "AND-OR-Graph-Search" del capítulo sobre algoritmos de búsqueda.
Los detalles de implementación se explicarán en breve.
- La decisión sobre la vinculación con cláusulas de Horn es lineal en cuanto al tamaño de la base de conocimientos:
<br>
Sorprendentemente, los algoritmos de encadenamiento hacia adelante y hacia atrás atraviesan cada elemento de la base de conocimientos como máximo una vez, lo que simplifica enormemente el problema.
<br>
<br>
La función `pl_fc_entails` implementa un encadenamiento directo para ver si una base de conocimiento `KB` implica un símbolo `q`.
<br>
Antes de continuar, tenga en cuenta que `pl_fc_entails` no utiliza una instancia de `KB` ordinaria.
La base de conocimientos aquí es una instancia de la clase `PropDefiniteKB`, derivada de la clase `PropKB`,
pero modificado para almacenar cláusulas definidas.
<br>
El principal punto de diferencia surge en la inclusión de un método auxiliar para `PropDefiniteKB` que devuelve una lista de cláusulas en KB que tienen un símbolo determinado `p` en su premisa.

In [39]:
psource(PropDefiniteKB.clauses_with_premise)

Ahora echemos un vistazo al algoritmo `pl_fc_entails`.

In [40]:
psource(pl_fc_entails)

La función acepta una base de conocimientos `KB` (una instancia de `PropDefiniteKB`) y una consulta `q` como entradas.
<br>
<br>
`count` almacena inicialmente el número de símbolos en la premisa de cada oración en la base de conocimiento.
<br>
La función auxiliar "conjunciones" separa una oración determinada en conjunciones.
<br>
`inferido` se inicializa como un dictamen predeterminado *booleano*.
Esto nos servirá más adelante para comprobar si hemos inferido todas las premisas de cada cláusula del orden del día.
<br>
`agenda` inicialmente almacena una lista de cláusulas que la base de conocimientos sabe que son verdaderas.
La función auxiliar `is_prop_symbol` comprueba si el símbolo dado es un símbolo lógico proposicional válido.
<br>
<br>
Ahora iteramos a través de la "agenda", apareciendo un símbolo "p" en cada iteración.
Si la consulta `q` es la misma que `p`, sabemos que la vinculación se cumple.
<br>
La agenda se procesa, reduciendo el `count` en uno por cada implicación con una premisa `p`.
Se añade una conclusión a la agenda cuando el "recuento" llega a cero. Esto significa que sabemos que todas las premisas de esa implicación particular son verdaderas.
<br>
`clauses_with_premise` es un método útil de la clase `PropKB`.
Devuelve una lista de cláusulas en la base de conocimientos que tienen "p" en su premisa.
<br>
<br>
Ahora que tenemos una idea de cómo funciona esta función, veamos algunos ejemplos de su uso, pero primero debemos definir nuestra base de conocimientos. Suponemos que sabemos que las siguientes cláusulas son verdaderas.

In [41]:
clauses = ['(B & F)==>E', 
           '(A & E & F)==>G', 
           '(B & C)==>F', 
           '(A & B)==>D', 
           '(E & F)==>H', 
           '(H & I)==>J',
           'A', 
           'B', 
           'C']

Ahora "contaremos" esta información a nuestra base de conocimientos.

In [42]:
definite_clauses_KB = PropDefiniteKB()
for clause in clauses:
    definite_clauses_KB.tell(expr(clause))

Ahora podemos comprobar si nuestra base de conocimientos incluye las siguientes consultas.

In [43]:
pl_fc_entails(definite_clauses_KB, expr('G'))

True

In [44]:
pl_fc_entails(definite_clauses_KB, expr('H'))

True

In [45]:
pl_fc_entails(definite_clauses_KB, expr('I'))

False

In [46]:
pl_fc_entails(definite_clauses_KB, expr('J'))

False

### Verificación efectiva del modelo proposicional

Los segmentos anteriores aclaran el procedimiento algorítmico para la verificación del modelo.
En este segmento, buscamos formas de hacerlos computacionalmente eficientes.
<br>
El problema que intentamos resolver se denomina convencionalmente problema de satisfacibilidad proposicional, abreviado problema SAT.
En términos sencillos, si existe un modelo que satisface una fórmula booleana determinada, la fórmula se denomina satisfactible.
<br>
El problema del SAT fue el primer problema que se demostró como _NP-completo_.
Las principales características de un problema NP-completo son:
- Dada una solución a tal problema, es fácil verificar si la solución resuelve el problema.
- El tiempo necesario para resolver realmente el problema utilizando cualquier algoritmo conocido aumenta exponencialmente con respecto al tamaño del problema.
<br>
<br>
Debido a estas propiedades, a menudo se aplican métodos heurísticos y de aproximación para encontrar soluciones a estos problemas.
<br>
Es extremadamente importante poder resolver problemas SAT a gran escala de manera eficiente porque
Muchos problemas combinatorios en informática pueden reducirse convenientemente a comprobar la satisfacibilidad de una oración proposicional bajo algunas restricciones.
<br>
Introduciremos dos nuevos algoritmos que realizan la verificación del modelo proposicional de una manera computacionalmente efectiva.
<br>


### 1. Algoritmo DPLL (Davis-Putnam-Logeman-Loveland)
Este algoritmo es muy similar a Backtracking-Search.
Enumera recursivamente posibles modelos en profundidad con las siguientes mejoras con respecto a algoritmos como `tt_entails`:
1. Terminación anticipada:
<br>
En ciertos casos, el algoritmo puede detectar el valor de verdad de una afirmación utilizando sólo un modelo parcialmente completado.
Por ejemplo, $(P\lor Q)\land(P\lor R)$ es verdadero si P es verdadero, independientemente de otras variables.
Esto reduce significativamente el espacio de búsqueda.
2. Heurística de símbolo puro:
<br>
Un símbolo que tiene el mismo signo (positivo o negativo) en todas las cláusulas se llama _símbolo puro_.
No es difícil ver que cualquier modelo satisfactorio tendrá símbolos puros asignados de manera que su cláusula principal se vuelva verdadera.
Por ejemplo, $(P\lor\neg Q)\land(\neg Q\lor\neg R)\land(R\lor P)$ tiene P y Q como símbolos puros
y para que la oración sea verdadera, P _tiene_ que ser verdadera y Q _tiene_ que ser falsa.
La heurística del símbolo puro simplifica un poco el problema.
3. Heurística de la cláusula unitaria:
<br>
En el contexto de DPLL, las cláusulas con un solo literal y las cláusulas con todos los literales _falsos_ menos uno se denominan cláusulas unitarias.
Si una cláusula es una cláusula unitaria, sólo puede satisfacerse asignando el valor necesario para que el último literal sea verdadero.
No tenemos otra opción.
<br>
Asignar una cláusula unitaria puede crear otra cláusula unitaria.
Por ejemplo, cuando P es falso, $(P\lor Q)$ se convierte en una cláusula unitaria, lo que hace que se asigne _verdadero_ a Q.
Una serie de asignaciones forzadas derivadas de cláusulas unitarias anteriores se denomina _propagación unitaria_.
De esta forma, esta heurística simplifica aún más el problema.
<br>
El algoritmo suele emplear otros trucos para ampliar la escala a problemas grandes.
Sin embargo, estos trucos están actualmente fuera del alcance de este cuaderno. Consulte la sección 7.6 del libro para obtener más detalles.
<br>
<br>
Echemos un vistazo al algoritmo.

In [47]:
psource(dpll)

El algoritmo utiliza las ideas descritas anteriormente para comprobar la satisfacibilidad de una oración en lógica proposicional.
Se llama a sí mismo de forma recursiva, simplificando el problema en cada paso. También utiliza las funciones auxiliares `find_pure_symbol` y `find_unit_clause` para llevar a cabo los pasos 2 y 3 anteriores.
<br>
La función auxiliar `dpll_satisfiable` convierte las cláusulas de entrada a _forma normal conjuntiva_ y llama a la función `dpll` con los parámetros correctos.

In [48]:
psource(dpll_satisfiable)

Veamos algunos ejemplos de uso.

In [49]:
A, B, C, D = expr('A, B, C, D')

In [50]:
dpll_satisfiable(A & B & ~C & D)

{A: True, B: True, C: False, D: True}

Este es un caso sencillo para resaltar que el algoritmo realmente funciona.

In [51]:
dpll_satisfiable((A & B) | (C & ~A) | (B & ~D))

{B: True, C: True, D: False}

Si un símbolo particular no está presente en la solución,
significa que la solución es independiente del valor de ese símbolo.
En este caso, la solución es independiente de A.

In [52]:
dpll_satisfiable(A |'<=>'| B)

{A: True, B: True}

In [53]:
dpll_satisfiable((A |'<=>'| B) |'==>'| (C & ~A))

{A: False, B: True, C: True}

In [54]:
dpll_satisfiable((A | (B & C)) |'<=>'| ((A | B) & (A | C)))

{B: True, C: True}

### 2. Algoritmo WalkSAT
Este algoritmo es muy similar a la escalada de colinas.
En cada iteración, el algoritmo selecciona una cláusula insatisfecha y voltea un símbolo en la cláusula.
Esto es similar a encontrar un estado vecino en el algoritmo `hill_climbing`.
<br>
El símbolo que se va a invertir se decide mediante una función de evaluación que cuenta el número de cláusulas insatisfechas.
A veces, los símbolos también se invierten aleatoriamente para evitar óptimos locales. Se requiere un equilibrio sutil entre la codicia y la aleatoriedad. Alternativamente, algunas versiones del algoritmo se reinician con una asignación aleatoria completamente nueva si no se ha encontrado una solución durante demasiado tiempo como forma de salir de los mínimos locales de números de cláusulas insatisfechas.
<br>
<br>
Echemos un vistazo al algoritmo.

In [55]:
psource(WalkSAT)

La función toma tres argumentos:
<br>
1. Las `cláusulas` que queremos satisfacer.
<br>
2. La probabilidad "p" de cambiar aleatoriamente un símbolo.
<br>
3. El número máximo de lanzamientos (`max_flips`) que se ejecutará el algoritmo. Si las cláusulas aún no se cumplen, el algoritmo devuelve "Ninguno" para indicar un error.
<br>
El algoritmo es idéntico en concepto a Hill Climbing y el código no es difícil de entender.
<br>
<br>
Veamos algunos ejemplos de uso.

In [56]:
A, B, C, D = expr('A, B, C, D')

In [57]:
WalkSAT([A, B, ~C, D], 0.5, 100)

{A: True, B: True, C: False, D: True}

Este es un caso sencillo para demostrar que el algoritmo converge.

In [58]:
WalkSAT([A & B, A & C], 0.5, 100)

{A: True, B: True, C: True}

In [59]:
WalkSAT([A & B, C & D, C & B], 0.5, 100)

{A: True, B: True, C: True, D: True}

In [60]:
WalkSAT([A & B, C | D, ~(D | B)], 0.5, 1000)

Éste no da ningún resultado porque WalkSAT no encontró ningún modelo en el que se cumplan estas cláusulas. Podemos resolver estas cláusulas para ver que juntas forman una contradicción y, por lo tanto, se supone que no tiene solución.

Un punto de diferencia entre este algoritmo y los algoritmos `dpll_satisfiable` es que ambos algoritmos toman entradas de manera diferente.
Para que WalkSAT tome oraciones completas como entrada,
Podemos escribir una función auxiliar que convierta la oración de entrada en forma conjuntiva normal y luego llame a WalkSAT con la lista de conjunciones de la forma CNF de la oración.

In [61]:
def WalkSAT_CNF(sentence, p=0.5, max_flips=10000):
    return WalkSAT(conjuncts(to_cnf(sentence)), 0, max_flips)

Ahora podemos llamar a `WalkSAT_CNF` y `DPLL_Satisfiable` con los mismos argumentos.

In [62]:
WalkSAT_CNF((A & B) | (C & ~A) | (B & ~D), 0.5, 1000)

{A: True, B: True, C: False, D: True}

¡Funciona!
<br>
Observe que la solución generada por WalkSAT no omite variables de las que no depende la oración.
Si la oración es independiente de una variable particular, la solución contiene un valor aleatorio para esa variable debido a la naturaleza estocástica del algoritmo.
<br>
<br>
Comparemos el tiempo de ejecución de WalkSAT y DPLL en algunos casos. Usaremos la magia `%%timeit` para hacer esto.

In [63]:
sentence_1 = A |'<=>'| B
sentence_2 = (A & B) | (C & ~A) | (B & ~D)
sentence_3 = (A | (B & C)) |'<=>'| ((A | B) & (A | C))

In [64]:
%%timeit
dpll_satisfiable(sentence_1)
dpll_satisfiable(sentence_2)
dpll_satisfiable(sentence_3)

1.55 ms ± 64.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [65]:
%%timeit
WalkSAT_CNF(sentence_1)
WalkSAT_CNF(sentence_2)
WalkSAT_CNF(sentence_3)

1.02 ms ± 6.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


En promedio, para casos solucionables, `WalkSAT` es bastante más rápido que `dpll` porque, para una pequeña cantidad de variables,
`WalkSAT` puede reducir significativamente el espacio de búsqueda.
Sin embargo, los resultados pueden ser diferentes para oraciones con más símbolos.
Siéntase libre de jugar con esto para comprender mejor las ventajas y desventajas de estos algoritmos.

### Plan SAT

En esta sección mostramos cómo hacer planes por inferencia lógica. La idea basica es muy simple. Incluye los siguientes tres pasos:
1. Construya una oración que incluya:
1. Una colección de afirmaciones sobre el estado inicial.
2. Los axiomas del estado sucesor para todas las acciones posibles en cada momento hasta un tiempo máximo t.
3. La afirmación de que el objetivo se logra en el momento t.
2. Presente la oración completa a un solucionador del SAT.
3. Suponiendo que se encuentra un modelo, extraiga del modelo aquellas variables que representan acciones y se les asigna como verdaderas. Juntos representan un plan para lograr los objetivos.


Echemos un vistazo al algoritmo.

In [66]:
psource(SAT_plan)

Veamos algunos ejemplos de su uso. Primero definimos una transición y luego llamamos `SAT_plan`.

In [67]:
transition = {'A': {'Left': 'A', 'Right': 'B'},
            'B': {'Left': 'A', 'Right': 'C'},
            'C': {'Left': 'B', 'Right': 'C'}}


print(SAT_plan('A', transition, 'C', 2)) 
print(SAT_plan('A', transition, 'B', 3))
print(SAT_plan('C', transition, 'A', 3))

None
['Right']
['Left', 'Left']


Hagamos lo mismo para otra transición.

In [68]:
transition = {(0, 0): {'Right': (0, 1), 'Down': (1, 0)},
            (0, 1): {'Left': (1, 0), 'Down': (1, 1)},
            (1, 0): {'Right': (1, 0), 'Up': (1, 0), 'Left': (1, 0), 'Down': (1, 0)},
            (1, 1): {'Left': (1, 0), 'Up': (0, 1)}}


print(SAT_plan((0, 0), transition, (1, 1), 4))

['Right', 'Down']


## Bases de conocimientos de lógica de primer orden: `FolKB`

La clase `FolKB` se puede utilizar para representar una base de conocimientos de oraciones lógicas de primer orden. Lo inicializaría y lo usaría de la misma manera que lo haría con `PropKB` excepto que las cláusulas son cláusulas definidas de primer orden. Veremos cómo escribir dichas cláusulas para crear una base de datos y consultarlas en las siguientes secciones.

## KB criminal
En esta sección creamos un `FolKB` basado en el siguiente párrafo.<br/>
<em>La ley dice que es un delito que un estadounidense venda armas a naciones hostiles. El país Nono, enemigo de Estados Unidos, tiene algunos misiles, y todos sus misiles le fueron vendidos por el coronel West, que es estadounidense.</em><br/>
El primer paso es extraer los hechos y convertirlos en cláusulas definidas de primer orden. Extraer los hechos únicamente a partir de los datos es una tarea desafiante. Afortunadamente, tenemos un párrafo pequeño y podemos realizar la extracción y conversión manualmente. Almacenaremos las cláusulas en una lista denominada apropiadamente "cláusulas".

In [69]:
clauses = []

<em>“... es un crimen que un estadounidense venda armas a naciones hostiles”</em><br/>
Las palabras clave que se deben buscar aquí son "crimen", "estadounidense", "vender", "arma" y "hostil". Usamos símbolos de predicados para darles significado.

* `Criminal(x)`: `x` es un criminal
* `American(x)`: `x` es un americano
* `Sells(x,y, z)`: `x` vende `y` a `z`
* `Arma(x)`: `x` es un arma
* `Hostile(x)`: `x` es una nación hostil

Combinémoslos ahora con nombres de variables apropiados para representar el significado de la oración. El criminal "x" es también el "x" estadounidense que vende armas "y" a "z", que es una nación hostil.

$\text{Americano}(x) \land \text{Arma}(y) \land \text{Vende}(x, y, z) \land \text{Hostil}(z) \implica \text{Criminal} (x)$

In [70]:
clauses.append(expr("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))

<em>"El país Nono, enemigo de América"</em><br/>
Ahora sabemos que Nono es un enemigo de Estados Unidos. Representamos a estas naciones usando los símbolos constantes "Nono" y "América". la relación enemiga se muestra utilizando el símbolo de predicado "Enemigo".

$\text{Enemigo}(\text{Nono}, \text{América})$

In [71]:
clauses.append(expr("Enemy(Nono, America)"))

<em>"Nono... tiene unos misiles"</em><br/>
Esto indica la existencia de algún misil propiedad de Nono. $\exists x \text{Propietario}(\text{Nono}, x) \land \text{Misil}(x)$. Invocamos la instanciación existencial para introducir una nueva constante "M1", que es el misil propiedad de Nono.

$\text{Propietario}(\text{Nono}, \text{M1}), \text{Misil}(\text{M1})$

In [72]:
clauses.append(expr("Owns(Nono, M1)"))
clauses.append(expr("Missile(M1)"))

<em>"Todos sus misiles le fueron vendidos por el coronel West"</em><br/>
Si Nono posee algo y se clasifica como un misil, entonces West se lo vendió a Nono.

$\text{Misil}(x) \land \text{Propietario}(\text{Nono}, x) \implica \text{Vende}(\text{Oeste}, x, \text{Nono})$

In [73]:
clauses.append(expr("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))

<em>"West, ¿quién es americano"</em><br/>
West es estadounidense.

$\text{Americano}(\text{Oeste})$

In [74]:
clauses.append(expr("American(West)"))

También sabemos, por nuestra comprensión del lenguaje, que los misiles son armas y que un enemigo de Estados Unidos se considera “hostil”.

$\text{Misil}(x) \implica \text{Arma}(x), \text{Enemigo}(x, \text{Estados Unidos}) \implica \text{Hostil}(x)$

In [75]:
clauses.append(expr("Missile(x) ==> Weapon(x)"))
clauses.append(expr("Enemy(x, America) ==> Hostile(x)"))

Ahora que hemos convertido la información en cláusulas definidas de primer orden, podemos crear nuestra base de conocimientos de lógica de primer orden.

In [76]:
crime_kb = FolKB(clauses)

La función auxiliar `subst` sustituye variables con valores dados en declaraciones lógicas de primer orden.
Esto será útil en algoritmos posteriores.
Su implementación es bastante simple y se explica por sí misma.

In [77]:
psource(subst)

A continuación se muestra un ejemplo de cómo se puede utilizar "subst".

In [78]:
subst({x: expr('Nono'), y: expr('M1')}, expr('Owns(x, y)'))

Owns(Nono, M1)

## Inferencia en lógica de primer orden
En esta sección veremos un algoritmo de encadenamiento hacia adelante y hacia atrás para "FolKB". Los dos algoritmos antes mencionados se basan en un proceso llamado <strong>unificación</strong>, un componente clave de todos los algoritmos de inferencia de primer orden.

### Unificación
A veces necesitamos encontrar sustituciones que hagan que diferentes expresiones lógicas parezcan idénticas. Este proceso, llamado unificación, se realiza mediante el algoritmo "unificar". Toma como entrada dos oraciones y devuelve un <em>unificador</em> para ellas, si existe. Un unificador es un diccionario que almacena las sustituciones necesarias para que dos oraciones sean idénticas. Lo hace unificando recursivamente los componentes de una oración, donde la unificación de un símbolo variable `var` con un símbolo constante `Const` es el mapeo `{var: Const}`. Veamos algunos ejemplos.

In [79]:
unify(expr('x'), 3)

{x: 3}

In [80]:
unify(expr('A(x)'), expr('A(B)'))

{x: B}

In [81]:
unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(y)'))

{x: Bella, y: Dobby}

En los casos en los que no existe una sustitución posible que unifique las dos oraciones la función devuelve `Ninguno`.

In [82]:
print(unify(expr('Cat(x)'), expr('Dog(Dobby)')))

None


También debemos tener cuidado de no utilizar involuntariamente el mismo nombre de variable. Unify los trata como una única variable, lo que le impide tomar valores múltiples.

In [83]:
print(unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(x)')))

None


### Algoritmo de encadenamiento directo
Consideramos el algoritmo simple de encadenamiento directo presentado en la <em>Figura 9.3</em>. Observamos cada regla en la base de conocimientos y vemos si se pueden cumplir las premisas. Esto se hace encontrando una sustitución que unifique cada una de las premisas con una cláusula en la "KB". Si conseguimos unificar las premisas, la conclusión (con la correspondiente sustitución) se añade al `KB`. Este proceso de inferencia se repite hasta que se pueda responder la consulta o hasta que no se puedan agregar nuevas oraciones. Probamos si la cláusula recién agregada se unifica con la consulta, en cuyo caso la sustitución obtenida por "unificar" es una respuesta a la consulta. Si nos quedamos sin oraciones para inferir, esto significa que la consulta fue un fracaso.

La función `fol_fc_ask` es un generador que genera todas las sustituciones que validan la consulta.

In [84]:
psource(fol_fc_ask)

Descubramos todas las naciones hostiles. Tenga en cuenta que sólo le dijimos al "KB" que Nono era un enemigo de Estados Unidos, no que fuera hostil.

In [85]:
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}]


El generador devolvió una única sustitución que dice que Nono es una nación hostil. Mira cómo después de agregar otra nación enemiga el generador devuelve dos sustituciones.

In [86]:
crime_kb.tell(expr('Enemy(JaJa, America)'))
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}, {x: JaJa}]


<strong><em>Nota</em>:</strong> `fol_fc_ask` realiza cambios en la `KB` agregándole oraciones.

### Algoritmo de encadenamiento hacia atrás
Este algoritmo trabaja hacia atrás desde el objetivo, encadenando reglas para encontrar hechos conocidos que respalden la prueba. Supongamos que "objetivo" es la consulta para la que queremos encontrar la sustitución. Encontramos reglas de la forma $\text{lhs} \implies \text{goal}$ en la `KB` e intentamos demostrar `lhs`. Puede haber varias cláusulas en la "KB" que proporcionen múltiples "lhs". Basta con probar sólo uno de ellos. Pero para probar una `lhs` se deben probar todas las conjunciones en las `lhs` de la cláusula. Esto lo hace similar a la búsqueda <em>Y/O</em>.

#### O
La parte <em>OR</em> del algoritmo proviene de nuestra elección de seleccionar cualquier cláusula de la forma $\text{lhs} \implies \text{goal}$. Al observar los "lhs" de todas las reglas cuyos "rhs" se unen con la "meta", obtenemos una sustitución que demuestra todas las conjunciones en los "lhs". Usamos `parse_definite_clause` para obtener `lhs` y `rhs` de una cláusula de la forma $\text{lhs} \implies \text{rhs}$. Para hechos atómicos, "lhs" es una lista vacía.

In [87]:
psource(fol_bc_or)

#### Y
El <em>AND</em> corresponde a demostrar todas las conjunciones en el `lhs`. Necesitamos encontrar una sustitución que pruebe cada <em>y</em> cláusula en la lista de conjuntos.

In [88]:
psource(fol_bc_and)

Ahora la función principal `fl_bc_ask` llama a `fol_bc_or` con la sustitución inicializada como vacía. El método `ask` de `FolKB` usa `fol_bc_ask` y recupera la primera sustitución devuelta por el generador para responder la consulta. Consultemos la base de conocimientos que creamos a partir de "cláusulas" para encontrar naciones hostiles.

In [89]:
# Rebuild KB because running fol_fc_ask would add new facts to the KB
crime_kb = FolKB(clauses)

In [90]:
crime_kb.ask(expr('Hostile(x)'))

{v_5: x, x: Nono}

Es posible que observe algunas variables nuevas en la sustitución. Se introducen para estandarizar los nombres de las variables y evitar problemas de denominación, como se explica en [Unification section](#Unification).

## Apéndice: La implementación de `|'==>'|`

Considere la `Expr` formada por esta sintaxis:

In [91]:
P |'==>'| ~Q

(P ==> ~Q)

¿Cuál es la sintaxis divertida `|'==>'|`? El truco es que "`|`" es simplemente el operador u normal de Python, por lo que es exactamente equivalente a esto:

In [92]:
(P | '==>') | ~Q

(P ==> ~Q)

En otras palabras, hay dos aplicaciones de los operadores or. Aquí está el primero:

In [93]:
P | '==>'

PartialExpr('==>', P)

Lo que sucede aquí es que el método `__or__` de `Expr` tiene un doble propósito. Si el lado derecho es otra `Expr` (o un número), entonces el resultado es una `Expr`, como en `(P | Q)`. Pero si el lado derecho es una cadena, entonces la cadena se toma como un operador y creamos un nodo en el árbol de sintaxis abstracta correspondiente a un `Expr` parcialmente lleno, uno donde conocemos el lado izquierdo. -lado es `P` y el operador es `==>`, pero aún no conocemos el lado derecho.

La clase `PartialExpr` tiene un método `__or__` que dice crear un nodo `Expr` con el lado derecho completado. Aquí podemos ver la combinación de `PartialExpr` con `Q` para crear un ` completo Expr`:

In [94]:
partial = PartialExpr('==>', P) 
partial | ~Q

(P ==> ~Q)

Este [trick](http://code.activestate.com/recipes/384122-infix-operators/) se debe a [trick](http://code.activestate.com/recipes/384122-infix-operators/), con una modificación de [trick](http://code.activestate.com/recipes/384122-infix-operators/),
quien sugirió usar una cuerda dentro de las barras o.

## Apéndice: La implementación de `expr`

¿Cómo "expr" analiza una cadena en un "Expr"? Resulta que hay dos trucos (además del truco de Jamitzky/Vedant):

1. Hacemos una sustitución de cadenas, reemplazando "`==>`" con "`|'==>'|`" (y lo mismo para otros operadores).
2. `Evaluamos` la cadena resultante en un entorno en el que cada identificador
está vinculado a un símbolo con ese identificador como "op".

En otras palabras,

In [95]:
expr('~(P & Q)  ==>  (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

es equivalente a hacer:

In [96]:
P, Q = symbols('P, Q')
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

Una cosa a tener en cuenta: esto coloca `==>` en el mismo nivel de precedencia que `"|"`, lo cual no es del todo correcto. Por ejemplo, obtenemos esto:

In [97]:
P & Q  |'==>'|  P | Q

(((P & Q) ==> P) | Q)

que probablemente no sea lo que queríamos decir; En caso de duda, ponga pares adicionales:

In [98]:
(P & Q)  |'==>'|  (P | Q)

((P & Q) ==> (P | Q))

## Ejemplos

In [99]:
from notebook import Canvas_fol_bc_ask
canvas_bc_ask = Canvas_fol_bc_ask('canvas_bc_ask', crime_kb, expr('Criminal(x)'))

# Autores

Este cuaderno de [Chirag Vartak](https://github.com/chiragvartak) y [Chirag Vartak](https://github.com/chiragvartak).

